# Dataset #

- Location: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
- Subject: US Accidents between 2016 and 2023

# Problem to Solve #

1. Problem 1: Given the weather forecast information create a model that will predict how many accidents will happen.
    - Reason: predict how many accidents will happen allows the local emergency services plan ahead 

2. Problem 2: Given the weather forecast information create a model that will predict the total impact for that day.
    - Note: impact = duration of the problems caused by the accidents
    - Reason: impact means that people was stuck on trafic loosing productive time. Reduce the number of accidents can increse the overal productivity of the population by reducing time-wated 

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
# load the dataset
accidents = pd.read_csv("../raw/US_Accidents_March23.csv")

In [3]:
print (accidents.shape)
accidents.head()

(7728394, 46)


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


# What Infor is Not Related to the Problem? #

1. Location Coordinates (Lat / Log)
    - End_Lat
    - End_Lng
    - Street

2. General Information about the accident
    - source: Source of raw accident data
    - ID: This is a unique identifier of the accident record.
    - description: hows a human provided description of the accident.
    - Country
    - Timezone
    - Airport_Code


3. Sunset - Twilight variations - Sunrise
    - Civil_Twilight: Shows the period of day (i.e. day or night) based on civil twilight.
    - Nautical_Twilight: Shows the period of day (i.e. day or night) based on nautical twilight.
    - Astronomical_Twilight: Shows the period of day (i.e. day or night) based on astronomical twilight.

        - Civil Twilight: This phase begins when the Sun is less than 6 degrees below the horizon. In the morning, it starts before sunrise, and in the evening, it begins at sunset. Civil twilight is the brightest form of twilight, providing enough natural sunlight for outdoor activities without artificial light. It’s often used for aviation, hunting, and street lighting regulations1.
        - Nautical Twilight: Occurring when the Sun is between 6 and 12 degrees below the horizon, nautical twilight is less bright than civil twilight. During this phase, artificial light is generally needed for outdoor activities. Nautical twilight has historical significance for sailors who used stars to navigate the seas. It’s also relevant for military planning1.
        - Astronomical Twilight: This phase happens when the Sun is between 12 and 18 degrees below the horizon. It’s the darkest form of twilight. Astronomers use it to define dawn and dusk. During astronomical twilight, celestial objects become visible, and the horizon remains clear. It’s essential for stargazing and astronomical observations

4. Miscelaneous
    - Zipcode


# Useful Infomratio for the Problem #

1. Accident Location
    - City
    - County
    - State

2. Impact of the accident
    - New Column: Duration (End_Time - Start_Time)
        - Start_time: Shows start time of the accident in local time zone.
        - End_Time: Shows end time of the accident in local time zone. End time here refers to when the impact of accident on traffic flow was dismissed.
    - Severity: Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).
    - Distance(mi): The length of the road extent affected by the accident in miles.

3. Accident Date and Accidcent Time
    - New Column -> Accident_Date (hh:00:00 -> start_time without date)
    - New Column -> Accident_Time (yyyy-mm-dd -> start_time without hour)
    - New Column -> is_weekend: True / False (calculated as: accident_weekday > 4)
    - New Column -> is_holiday: True / False
    - New Column -> accident_weekday: day of the week of the accident (0: Monday to 6: Sunday)
    - Start_Time: Shows start time of the accident in local time zone.
    - End_Time: Shows end time of the accident in local time zone. End time here refers to when the impact of accident on traffic flow was dismissed.
   
4. Weather Information

    - Weather_Timestamp: Shows the time-stamp of weather observation record (in local time).
    - Temperature(F)
    - Wind_Chill(F)
    - Humidity(%)
    - Pressure(in)
    - Visibility(mi): Shows visibility (in miles).
    - Wind_Direction
    - Wind_Speed(mph)
    - Precipitation(in)
    - Weather_Condition: Shows the weather condition (rain, snow, thunderstorm, fog, etc.)

5. Points of Interest
    -  Amenity: A POI annotation which indicates presence of amenity in a nearby location.
    - Bump: A POI annotation which indicates presence of speed bump or hump in a nearby location.
    -  Crossing: A POI annotation which indicates presence of crossing in a nearby location.
    - Give_away: A POI annotation which indicates presence of give_way in a nearby location
    - Junction: A POI annotation which indicates presence of junction in a nearby location.
    - No_Exit: A POI annotation which indicates presence of no_exit in a nearby location.
    - Railway: A POI annotation which indicates presence of railway in a nearby location.
    - Roundabout: A POI annotation which indicates presence of roundabout in a nearby location.
    - Station: A POI annotation which indicates presence of station in a nearby location.
    - Stop: A POI annotation which indicates presence of stop in a nearby location.
    - Traffic_Calming: A POI annotation which indicates presence of traffic_calming in a nearby location.
    - Traffic_Signal: A POI annotation which indicates presence of traffic_signal in a nearby location.
    - Turning_Loop: A POI annotation which indicates presence of turning_loop in a nearby location.

6. Day Night
    - sunrise_sunset

In [4]:
# create a new DataFrame with accidents from Los Angeles
accidents_temp = pd.DataFrame(accidents.loc[(accidents.City == 'Los Angeles')])
accidents_temp.shape


(156491, 46)

In [5]:
#############################################################################
# drop the columns that will not be used
#############################################################################

# 'End_Lat','End_Lng'
# 'source','ID','description','Country','Timezone','Airport_Code'
# 'Civil_Twilight','Nautical_Twilight','Astronomical_Twilight'
# 'Zipcode'


accidents_temp.drop(columns=['End_Lat','End_Lng','Street'], inplace=True)
accidents_temp.drop(columns=['Source','ID','Description','Country','Timezone','Airport_Code'], inplace=True)
accidents_temp.drop(columns=['Civil_Twilight','Nautical_Twilight','Astronomical_Twilight'], inplace=True)                            
accidents_temp.drop(columns=['Zipcode'], inplace=True) 

accidents_temp.shape

accidents_temp.head()

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),City,County,State,Weather_Timestamp,...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
42866,2,2016-06-21 10:46:30,2016-06-21 11:27:00,34.078926,-118.289040,0.0,Los Angeles,Los Angeles,CA,2016-06-21 10:47:00,...,True,False,False,False,False,False,False,False,False,Day
42867,3,2016-06-21 10:49:21,2016-06-21 11:34:21,34.091179,-118.239471,0.0,Los Angeles,Los Angeles,CA,2016-06-21 10:47:00,...,False,False,False,False,False,False,False,False,False,Day
42881,3,2016-06-21 10:51:45,2016-06-21 11:36:45,34.037239,-118.309074,0.0,Los Angeles,Los Angeles,CA,2016-06-21 10:47:00,...,False,False,False,False,True,False,False,False,False,Day
42883,3,2016-06-21 10:56:24,2016-06-21 11:34:00,34.027458,-118.274490,0.0,Los Angeles,Los Angeles,CA,2016-06-21 10:47:00,...,False,False,False,False,False,False,False,False,False,Day
42898,3,2016-06-21 11:30:46,2016-06-21 12:00:46,33.947544,-118.279434,0.0,Los Angeles,Los Angeles,CA,2016-06-21 11:53:00,...,False,False,False,False,False,False,False,False,False,Day


In [6]:
#################################################################
# Solving Missing Data Points from Data Frame
#################################################################

missing_values_count = accidents_temp.isnull().sum()

missing_values_count[0:]

Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
Distance(mi)             0
City                     0
County                   0
State                    0
Weather_Timestamp      470
Temperature(F)        1105
Wind_Chill(F)        52064
Humidity(%)           1180
Pressure(in)           965
Visibility(mi)         684
Wind_Direction        2431
Wind_Speed(mph)      23989
Precipitation(in)    47471
Weather_Condition      650
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal           0
Turning_Loop             0
Sunrise_Sunset           0
dtype: int64

In [7]:
missing_values_count [missing_values_count[0:] != 0]

Weather_Timestamp      470
Temperature(F)        1105
Wind_Chill(F)        52064
Humidity(%)           1180
Pressure(in)           965
Visibility(mi)         684
Wind_Direction        2431
Wind_Speed(mph)      23989
Precipitation(in)    47471
Weather_Condition      650
dtype: int64

In [8]:
accidents_temp.Start_Time = pd.to_datetime(accidents_temp.Start_Time , format = "mixed")

In [9]:

# Accident Date (no hour)
accidents_temp['Accident_Date'] = accidents_temp.Start_Time.dt.date


In [10]:

accidents_temp['accidnet_weekday'] = accidents_temp.Start_Time.dt.weekday
accidents_temp['accidnet_weekday']

# 0: Monday
# 1: Tuesday
# 2: Wednesday
# 3: Thursday
# 4: Friday
# 5: Saturday
# 6: Sunday


42866      1
42867      1
42881      1
42883      1
42898      1
          ..
7727947    3
7728357    4
7728361    4
7728364    4
7728376    4
Name: accidnet_weekday, Length: 156491, dtype: int32

In [11]:
# 0: Monday
# 1: Tuesday
# 2: Wednesday
# 3: Thursday
# 4: Friday
# 5: Saturday
# 6: Sunday
 
accidents_temp['is_weekend'] = accidents_temp.Start_Time.dt.weekday > 4
accidents_temp['is_weekend'].value_counts(False)



is_weekend
False    121480
True      35011
Name: count, dtype: int64

In [12]:
%pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [13]:
import holidays
ca_holidays = holidays.US(state='CA')

accidents_temp['is_holiday'] = accidents_temp['Start_Time'].dt.date.apply(lambda x: x in ca_holidays)



In [14]:

accidents_temp['is_holiday'].value_counts(False)


is_holiday
False    151813
True       4678
Name: count, dtype: int64

In [15]:
# Accident Hour (no date)
accidents_temp['Accident_Time'] = pd.to_datetime(accidents_temp.Start_Time , format = "mixed")
accidents_temp['Accident_Time'] = accidents_temp['Accident_Time'].dt.hour

In [16]:
accidents_temp['Accident_Time']

42866      10
42867      10
42881      10
42883      10
42898      11
           ..
7727947    17
7728357     4
7728361    12
7728364    13
7728376    15
Name: Accident_Time, Length: 156491, dtype: int32

In [17]:
# Temperature(F)
# Replace the null values by the average temperature

print ("Temperature Null:", accidents_temp['Temperature(F)'].map(lambda p: pd.isnull(p)).sum() )
print ("Temperature Not Null:", accidents_temp['Temperature(F)'].notnull().sum())
print ("Temperature Mean:",accidents_temp['Temperature(F)'].mean())

# Temperature Null: 1105
# Temperature Not Null: 155386
# Mean: 65.65538465498824


Temperature Null: 1105
Temperature Not Null: 155386
Temperature Mean: 65.65538465498824


In [18]:
# Initialize KNNImputer with the desired number of neighbors
imputer = KNNImputer(n_neighbors=3)

In [19]:
# Temperature(F)
# use the imputer to resolve

accidents_temp = accidents_temp.reset_index()

# # Convert the Series to a DataFrame
temperature_df = accidents_temp[['Temperature(F)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(temperature_df)

# Convert the result back to a DataFrame
accidents_temp['Temperature(F)'] = pd.DataFrame(df_imputed, columns=['Temperature(F)'])

###########################################################
###########################################################


In [20]:
missing_values_count = accidents_temp.isnull().sum()

missing_values_count[0:]

index                    0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
Distance(mi)             0
City                     0
County                   0
State                    0
Weather_Timestamp      470
Temperature(F)           0
Wind_Chill(F)        52064
Humidity(%)           1180
Pressure(in)           965
Visibility(mi)         684
Wind_Direction        2431
Wind_Speed(mph)      23989
Precipitation(in)    47471
Weather_Condition      650
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal           0
Turning_Loop             0
Sunrise_Sunset           0
Accident_Date            0
accidnet_weekday         0
is_weekend               0
i

In [21]:

'''

temperature_mean = accidents_temp['Temperature(F)'].mean()
temperature_rolling_mean = accidents_temp['Temperature(F)'].rolling(window=3, min_periods=1).mean()
accidents_temp['Temperature(F)'] = np.where (
                                            pd.isnull(accidents_temp['Temperature(F)']) 
                                            , temperature_rolling_mean
                                            , accidents_temp['Temperature(F)'] )

'''

"\n\ntemperature_mean = accidents_temp['Temperature(F)'].mean()\ntemperature_rolling_mean = accidents_temp['Temperature(F)'].rolling(window=3, min_periods=1).mean()\naccidents_temp['Temperature(F)'] = np.where (\n                                            pd.isnull(accidents_temp['Temperature(F)']) \n                                            , temperature_rolling_mean\n                                            , accidents_temp['Temperature(F)'] )\n\n"

In [22]:
# Weather_Timestamp
# when Weather_Timestamp is null replace by the start_time

accidents_temp['Weather_Timestamp'] = np.where (
                                            pd.isnull(accidents_temp['Weather_Timestamp']) 
                                            , accidents_temp['Start_Time']
                                            , accidents_temp['Weather_Timestamp'] )

In [23]:
# Humidity(%)
# when humidity is null, add humidity mean


print ("Humidity Null:", accidents_temp['Humidity(%)'].map(lambda p: pd.isnull(p)).sum() )
print ("Humidity Not Null:", accidents_temp['Humidity(%)'].notnull().sum())
print ("Humidity Mean:",accidents_temp['Humidity(%)'].mean())

'''
humidity_rolling_mean = accidents_temp['Humidity(%)'].rolling(window=3, min_periods=1).mean()

accidents_temp['Humidity(%)'] = np.where ( pd.isnull(accidents_temp['Humidity(%)']) 
                                            , humidity_rolling_mean
                                            , accidents_temp['Humidity(%)'] )
'''


Humidity Null: 1180
Humidity Not Null: 155311
Humidity Mean: 60.53426994868361


"\nhumidity_rolling_mean = accidents_temp['Humidity(%)'].rolling(window=3, min_periods=1).mean()\n\naccidents_temp['Humidity(%)'] = np.where ( pd.isnull(accidents_temp['Humidity(%)']) \n                                            , humidity_rolling_mean\n                                            , accidents_temp['Humidity(%)'] )\n"

In [24]:
# Humidity(%)
# use the imputer to resolve

# # Convert the Series to a DataFrame
humidity_df = accidents_temp[['Humidity(%)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(humidity_df)

# Convert the result back to a DataFrame
accidents_temp['Humidity(%)'] = pd.DataFrame(df_imputed, columns=['Humidity(%)'])

In [25]:
# Pressure(in)
# when Pressure(in) is null replace by Pressure(in)' mean

print ("Pressure Null:", accidents_temp['Pressure(in)'].map(lambda p: pd.isnull(p)).sum() )
print ("Pressure Not Null:", accidents_temp['Pressure(in)'].notnull().sum())
print ("Pressure Mean:",accidents_temp['Pressure(in)'].mean())

'''
pressure_mean = accidents_temp['Pressure(in)'].mean()

accidents_temp['Pressure(in)'] = np.where ( pd.isnull(accidents_temp['Pressure(in)']) 
                                            , pressure_mean
                                            , accidents_temp['Pressure(in)'] )
'''


Pressure Null: 965
Pressure Not Null: 155526
Pressure Mean: 29.852869938145385


"\npressure_mean = accidents_temp['Pressure(in)'].mean()\n\naccidents_temp['Pressure(in)'] = np.where ( pd.isnull(accidents_temp['Pressure(in)']) \n                                            , pressure_mean\n                                            , accidents_temp['Pressure(in)'] )\n"

In [26]:
# Pressure(in)
# use the imputer to resolve

# # Convert the Series to a DataFrame
pressure_df = accidents_temp[['Pressure(in)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(pressure_df)

# Convert the result back to a DataFrame
accidents_temp['Pressure(in)'] = pd.DataFrame(df_imputed, columns=['Pressure(in)'])

In [27]:
# Visibility(mi)
# when visibility is null, replace by the mean
# 
  
print ("Visibility Null:", accidents_temp['Visibility(mi)'].map(lambda p: pd.isnull(p)).sum() )
print ("Visibility Not Null:", accidents_temp['Visibility(mi)'].notnull().sum())
print ("Visibility Mean:",accidents_temp['Visibility(mi)'].mean())

# Visibility Null: 684
# Visibility Not Null: 155807
# Visibility Mean: 9.099849878375169

'''
visibility_mean = accidents_temp['Visibility(mi)'].mean()

accidents_temp['Visibility(mi)'] = np.where ( pd.isnull(accidents_temp['Visibility(mi)']) 
                                            , visibility_mean
                                            , accidents_temp['Visibility(mi)'] )
'''


Visibility Null: 684
Visibility Not Null: 155807
Visibility Mean: 9.099849878375169


"\nvisibility_mean = accidents_temp['Visibility(mi)'].mean()\n\naccidents_temp['Visibility(mi)'] = np.where ( pd.isnull(accidents_temp['Visibility(mi)']) \n                                            , visibility_mean\n                                            , accidents_temp['Visibility(mi)'] )\n"

In [28]:
# Visibility(mi)
# use the imputer to resolve

# # Convert the Series to a DataFrame
visibility_df = accidents_temp[['Visibility(mi)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(visibility_df)

# Convert the result back to a DataFrame
accidents_temp['Visibility(mi)'] = pd.DataFrame(df_imputed, columns=['Visibility(mi)'])

In [29]:
# Wind_Speed(mph)
# when wind_speed is null, replace by the mean

print ("Wind_Speed Null:", accidents_temp['Wind_Speed(mph)'].map(lambda p: pd.isnull(p)).sum() )
print ("Wind_Speed Not Null:", accidents_temp['Wind_Speed(mph)'].notnull().sum())
print ("Wind_Speed Mean:",accidents_temp['Wind_Speed(mph)'].mean())

# Wind_Speed Null: 23989
# Wind_Speed Not Null: 132502
# Wind_Speed Mean: 3.9225702253550887

'''
wind_speed_mean = accidents_temp['Wind_Speed(mph)'].mean()

accidents_temp['Wind_Speed(mph)'] = np.where ( pd.isnull(accidents_temp['Wind_Speed(mph)']) 
                                            , wind_speed_mean
                                            , accidents_temp['Wind_Speed(mph)'] )
'''


Wind_Speed Null: 23989
Wind_Speed Not Null: 132502
Wind_Speed Mean: 3.9225702253550887


"\nwind_speed_mean = accidents_temp['Wind_Speed(mph)'].mean()\n\naccidents_temp['Wind_Speed(mph)'] = np.where ( pd.isnull(accidents_temp['Wind_Speed(mph)']) \n                                            , wind_speed_mean\n                                            , accidents_temp['Wind_Speed(mph)'] )\n"

In [30]:
# Wind_Speed(mph)
# use the imputer to resolve

# # Convert the Series to a DataFrame
wind_speed_df = accidents_temp[['Wind_Speed(mph)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(wind_speed_df)

# Convert the result back to a DataFrame
accidents_temp['Wind_Speed(mph)'] = pd.DataFrame(df_imputed, columns=['Wind_Speed(mph)'])

In [31]:
# Precipitation(in)
# when precipitation is null, replace by the mean

print ("Precipitation Null:", accidents_temp['Precipitation(in)'].map(lambda p: pd.isnull(p)).sum() )
print ("Precipitation Not Null:", accidents_temp['Precipitation(in)'].notnull().sum())
print ("Precipitation Mean:",accidents_temp['Precipitation(in)'].mean())

# Precipitation Null: 47471
# Precipitation Not Null: 109020
# Precipitation Mean: 0.0034345074298293894

'''
precipitation_mean = accidents_temp['Precipitation(in)'].mean()

accidents_temp['Precipitation(in)'] = np.where ( pd.isnull(accidents_temp['Precipitation(in)']) 
                                            , precipitation_mean
                                            , accidents_temp['Precipitation(in)'] )
'''


Precipitation Null: 47471
Precipitation Not Null: 109020
Precipitation Mean: 0.0034345074298293894


"\nprecipitation_mean = accidents_temp['Precipitation(in)'].mean()\n\naccidents_temp['Precipitation(in)'] = np.where ( pd.isnull(accidents_temp['Precipitation(in)']) \n                                            , precipitation_mean\n                                            , accidents_temp['Precipitation(in)'] )\n"

In [32]:
# Precipitation(in)
# use the imputer to resolve

# # Convert the Series to a DataFrame
precipitation_df = accidents_temp[['Precipitation(in)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(precipitation_df)

# Convert the result back to a DataFrame
accidents_temp['Precipitation(in)'] = pd.DataFrame(df_imputed, columns=['Precipitation(in)'])

In [33]:
# Wind_Chill(F)
# use the imputer to resolve

# Convert the Series to a DataFrame
wind_chill_df = accidents_temp[['Wind_Chill(F)']]

# Fit and transform the DataFrame
df_imputed = imputer.fit_transform(wind_chill_df)

# Convert the result back to a DataFrame
accidents_temp['Wind_Chill(F)'] = pd.DataFrame(df_imputed, columns=['Wind_Chill(F)'])



In [34]:
accidents_temp['Wind_Direction']

0         Variable
1         Variable
2         Variable
3         Variable
4              ESE
            ...   
156486          SW
156487        CALM
156488        CALM
156489        CALM
156490           W
Name: Wind_Direction, Length: 156491, dtype: object

In [35]:
# Wind_Direction
# To replace missing values in a Pandas DataFrame with the value from the row below

print ("Wind Direction Null:", accidents_temp['Wind_Direction'].isnull().sum())
print ("Wind Direction NOT Null:", accidents_temp['Wind_Direction'].notnull().sum())
print ("---------------------------------------------------------------------------------------")

accidents_temp['Wind_Direction'] = np.where ( pd.isnull(accidents_temp['Wind_Direction']) 
                                            , accidents_temp['Wind_Direction'].bfill(axis='rows')
                                            , accidents_temp['Wind_Direction'] )

print ("Wind Direction Null:", accidents_temp['Wind_Direction'].isnull().sum())
print ("Wind Direction NOT Null:", accidents_temp['Wind_Direction'].notnull().sum())

Wind Direction Null: 2431
Wind Direction NOT Null: 154060
---------------------------------------------------------------------------------------
Wind Direction Null: 0
Wind Direction NOT Null: 156491


In [36]:
# How many Weather Conditions are in the DataSet?
#
# Use set() to eliminate duplicate values in column 'Weather_Condition'
unique_values_set = set(accidents['Weather_Condition'])

# Print the unique values
print (pd.DataFrame(unique_values_set))

                                0
0                             NaN
1                            Mist
2             Heavy Sleet / Windy
3                            Rain
4       Light Snow Shower / Windy
..                            ...
140            Wintry Mix / Windy
141             N/A Precipitation
142        Light Freezing Drizzle
143           Light Freezing Rain
144  Light Thunderstorms and Rain

[145 rows x 1 columns]


In [37]:
# Weather_Condition
# To replace missing values in a Pandas DataFrame with the value from the row below

print ("Weather Condition Null:", accidents_temp['Weather_Condition'].isnull().sum())
print ("Weather Condition NOT Null:", accidents_temp['Weather_Condition'].notnull().sum())
print ("---------------------------------------------------------------------------------------")

accidents_temp['Weather_Condition'] = np.where ( pd.isnull(accidents_temp['Weather_Condition']) 
                                            , accidents_temp['Weather_Condition'].bfill(axis='rows')
                                            , accidents_temp['Weather_Condition'] )

print ("Weather Condition Null:", accidents_temp['Weather_Condition'].isnull().sum())
print ("Weather Condition NOT Null:", accidents_temp['Weather_Condition'].notnull().sum())



Weather Condition Null: 650
Weather Condition NOT Null: 155841
---------------------------------------------------------------------------------------
Weather Condition Null: 0
Weather Condition NOT Null: 156491


In [38]:
accidents_temp.head()

,index,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),City,County,State,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Accident_Date,accidnet_weekday,is_weekend,is_holiday,Accident_Time
0,42866,2,2016-06-21 10:46:30,2016-06-21 11:27:00,34.078926,-118.289040,0.0,Los Angeles,Los Angeles,CA,...,False,False,False,False,Day,2016-06-21,1,False,False,10
1,42867,3,2016-06-21 10:49:21,2016-06-21 11:34:21,34.091179,-118.239471,0.0,Los Angeles,Los Angeles,CA,...,False,False,False,False,Day,2016-06-21,1,False,False,10
2,42881,3,2016-06-21 10:51:45,2016-06-21 11:36:45,34.037239,-118.309074,0.0,Los Angeles,Los Angeles,CA,...,False,False,False,False,Day,2016-06-21,1,False,False,10
3,42883,3,2016-06-21 10:56:24,2016-06-21 11:34:00,34.027458,-118.274490,0.0,Los Angeles,Los Angeles,CA,...,False,False,False,False,Day,2016-06-21,1,False,False,10
4,42898,3,2016-06-21 11:30:46,2016-06-21 12:00:46,33.947544,-118.279434,0.0,Los Angeles,Los Angeles,CA,...,False,False,False,False,Day,2016-06-21,1,False,False,11


In [39]:
accidents_temp.shape

(156491, 39)

In [40]:
missing_values_count = accidents_temp.isnull().sum()

missing_values_count[0:]

index                0
Severity             0
Start_Time           0
End_Time             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
City                 0
County               0
State                0
Weather_Timestamp    0
Temperature(F)       0
Wind_Chill(F)        0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
Accident_Date        0
accidnet_weekday     0
is_weekend           0
is_holiday           0
Accident_Time        0
dtype: int64

In [41]:
accidents_los_angeles = pd.DataFrame()

accidents_los_angeles.head()

""


In [42]:
##########################################################
# Build Data Frame with data agregated by day
##########################################################

accidents_los_angeles['Accident_Date'] = accidents_temp['Accident_Date'].unique()

In [43]:
# Weather Condition
# get the most common weather condition of the day and use it

weather_counts = accidents_temp.groupby(['Accident_Date', 'Weather_Condition']).size().reset_index(name='counts')
weather = weather_counts.loc[weather_counts.groupby('Accident_Date')['counts'].idxmax()]

# Merge the weather condition into the DataFrame
accidents_los_angeles = pd.merge(weather , accidents_los_angeles , on='Accident_Date')

accidents_los_angeles = accidents_los_angeles.drop('counts' , axis=1)

In [44]:
# Temperature 
# get the average of the day and use it

mean_temperature = accidents_temp.groupby('Accident_Date')['Temperature(F)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(mean_temperature , accidents_los_angeles , on='Accident_Date')



In [45]:
# Severity 
# get the average of the day and use it

mean_severity = accidents_temp.groupby('Accident_Date')['Severity'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(mean_severity , accidents_los_angeles , on='Accident_Date')


In [46]:
# Humidity(%) 
# get the average of the day and use it

mean_humidity = accidents_temp.groupby('Accident_Date')['Humidity(%)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles ,mean_humidity , on='Accident_Date')

In [47]:
# Pressure(in)
# get the average of the day and use it

mean_pressure = accidents_temp.groupby('Accident_Date')['Pressure(in)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles , mean_pressure , on='Accident_Date')

In [48]:
# Visibility(mi)
# get the average of the day and use it

mean_visibility = accidents_temp.groupby('Accident_Date')['Visibility(mi)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles , mean_visibility , on='Accident_Date')

In [49]:
# Wind_Speed(mph)
# get the average of the day and use it

mean_wind_speed = accidents_temp.groupby('Accident_Date')['Wind_Speed(mph)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles , mean_wind_speed , on='Accident_Date')

In [50]:
# Precipitation(in)
# get the average of the day and use it

mean_precipitation = accidents_temp.groupby('Accident_Date')['Precipitation(in)'].mean().reset_index()

# merge the temperature into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles , mean_precipitation , on='Accident_Date')

In [51]:
# Number of accidents per day
#

accidents_per_day = accidents_temp.groupby('Accident_Date').size().reset_index(name = "total_accidents")

# merge the accidents into the DataFrame
accidents_los_angeles = pd.merge(accidents_los_angeles , accidents_per_day , on='Accident_Date')


In [52]:
print (accidents_los_angeles.head())
print (accidents_los_angeles.shape)

  Accident_Date  Severity  Temperature(F) Weather_Condition  Humidity(%)  \
0    2016-03-22  2.400000       63.040000             Clear    26.000000   
1    2016-03-23  2.529412       68.491176             Clear    29.794118   
2    2016-03-24  2.485294       68.372059             Clear    33.544118   
3    2016-03-25  2.533333       65.831111             Clear    56.088889   
4    2016-03-26  2.736842       64.394737             Clear    70.631579   

   Pressure(in)  Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  \
0     30.016000       10.000000         9.680000           0.003435   
1     30.114559       10.000000         5.885718           0.003435   
2     30.008235        9.870588         5.265365           0.003435   
3     29.824444       10.000000         4.864368           0.003435   
4     29.853158        9.842105         6.060947           0.003435   

   total_accidents  
0                5  
1               68  
2               68  
3               45  
4          

In [53]:
# Export DataFrame to CSV


# data agreagated by day
accidents_los_angeles.to_csv("./Los_Angeles_Accidents_Agreagated_2016_2023.csv", index=False)

In [54]:
# raw data treated
accidents_temp.to_csv("./Los_Angeles_Accidents_Complete_2016_2023.csv", index=False)